In [50]:
import requests
from pprint import pprint,pp
def pprint_response(r):
    print(r.status_code)
    pp(dict(r.headers))
    print()
    pp(r.json())
    print(len(r.json()))
API_KEY="e967b89f3f884bccea030b81afba466eb4008a031255dfd9b3aa2d9aa0d03739"
u1 = {'name': 'Eva', 'email': 'eva@gmail.com', 'gender': 'female', 'status': 'inactive'}
u2 = {'name': 'Ana', 'email': 'ana2@gmail.com', 'gender': 'female', 'status': 'active'}
u3 = {'name': 'Pepe', 'email': 'pepe@gmail.com', 'gender': 'female', 'status': 'inactive'}
URL="https://gorest.co.in/"
def inserta_usuarios(datos, token):
    """ Inserta todos los usuarios de la lista y devuelve True si todos han sido insertados correctamente """
    url=URL+"/public/v2/users"
    contador=0
    for user in datos:
        response=requests.post(url,headers={'Authorization':f'Bearer {token}'},
                            data=user)
        print(response.status_code)
        if response.status_code==201:
            contador+=1
    return contador
    
print(inserta_usuarios([u1,u2,u3],API_KEY))

422
201
422
1


In [ ]:
def get_ident_email(email, token):
    """ Devuelve el identificador del usuario cuyo email sea *exactamente* el pasado como parámetro.
        En caso de que ese usuario no exista devuelve None """
    url=URL+"/public/v2/users"
    response=requests.get(url,headers={'Authorization':f'Bearer {token}'})
    users=response.json()
    for user in users:
        if user["email"]==email:
            return user["id"]
    return None

print(get_ident_email("ana2@gmail.com",API_KEY))
print(get_ident_email("Buenos_dias@gmail.com",API_KEY))

In [48]:
def borra_usuario(email, token):
    """ Elimina el usuario cuyo email sea *exactamente* el pasado como parámetro. En caso de éxito en el
        borrado devuelve True. Si no existe tal usuario devolverá False """
    id=get_ident_email(email,token)
    if(id is None):
        return False
    url=f'{URL}/public/v2/users/{id}'
    response=requests.delete(url,headers={'Authorization':f'Bearer {token}'})
    return response.status_code==204
url=URL+"/public/v2/users"
response=requests.get(url,headers={'Authorization':f'Bearer {API_KEY}'})
print(response.json())
print(borra_usuario("ana2@gmail.com",API_KEY))

[{'id': 7491601, 'name': 'Ana', 'email': 'ana2@gmail.com', 'gender': 'female', 'status': 'active'}, {'id': 7482875, 'name': 'Bharat Talwar IV', 'email': 'talwar_iv_bharat@collins.example', 'gender': 'female', 'status': 'inactive'}, {'id': 7482874, 'name': 'Prof. Amrit Kakkar', 'email': 'amrit_prof_kakkar@powlowski.test', 'gender': 'male', 'status': 'active'}, {'id': 7482872, 'name': 'Gov. Daksha Tagore', 'email': 'tagore_daksha_gov@moen.test', 'gender': 'male', 'status': 'inactive'}, {'id': 7482871, 'name': 'Gautama Trivedi', 'email': 'gautama_trivedi@herman.test', 'gender': 'male', 'status': 'active'}, {'id': 7482869, 'name': 'Dhanalakshmi Johar', 'email': 'johar_dhanalakshmi@thiel.example', 'gender': 'male', 'status': 'inactive'}, {'id': 7482868, 'name': 'Rev. Ajeet Naik', 'email': 'naik_rev_ajeet@okeefe.test', 'gender': 'female', 'status': 'active'}, {'id': 7482867, 'name': 'Dr. Veda Mahajan', 'email': 'veda_dr_mahajan@mante-beier.example', 'gender': 'male', 'status': 'active'}, {'i

In [55]:
#suponiendo que no se pueden añadir dos tareas con el mismo nombre
def inserta_todo(email, token, title, due_on, status='pending'):
    """ Inserta un nuevo ToDo para el usuario con email exactamente igual al pasado. Si el ToDo ha sido insertado
        con éxito devolverá True, en otro caso devolverá False """
    id=get_ident_email(email,token)
    if(id is None):
        return False
    url=f'{URL}/public/v2/users/{id}'
    response=requests.get(url,headers={'Authorization':f'Bearer {token}'})
    user=response.json()
    lista_tareas= user.get("tareas",[])
    print (lista_tareas)
    lista_tareas.append({"due_on":due_on,"status":status,"title":title})
    print (lista_tareas)
    response2=requests.patch(url,headers={'Authorization':f'Bearer {token}'},
                            data={"tareas":lista_tareas})
    pprint_response(response2)
    r=requests.get(url,headers={'Authorization':f'Bearer {token}'})
    print(r.json())
inserta_todo("ana2@gmail.com",API_KEY,"Todo 2","2034-07-28 11:3",'pending')

[]
[{'due_on': '2034-07-28 11:3', 'status': 'pending', 'title': 'Todo 2'}]
200
{'Date': 'Fri, 25 Oct 2024 21:07:55 GMT',
 'Content-Type': 'application/json; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Cache-Control': 'max-age=0, private, must-revalidate',
 'etag': 'W/"6fc544103f0ff2c0aed18eb1702959a4"',
 'referrer-policy': 'strict-origin-when-cross-origin',
 'vary': 'Origin',
 'x-content-type-options': 'nosniff',
 'x-download-options': 'noopen',
 'x-frame-options': 'SAMEORIGIN',
 'x-permitted-cross-domain-policies': 'none',
 'x-ratelimit-limit': '90',
 'x-ratelimit-remaining': '88',
 'x-ratelimit-reset': '2',
 'x-request-id': '8cb539b4-f1fc-40cd-bf27-dc267c36e34f',
 'x-runtime': '0.071812',
 'x-xss-protection': '0',
 'cf-cache-status': 'DYNAMIC',
 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=vHTq4k8aKi2WR7yuhQTxBtzbWujn%2FMHYP%2FH9%2BmT8M9Tz36iilJNUpLl%2FXTe0S0cUltYalmGzqM5Js8Xyns%2Fw3iGqEIEf%2F38cea2%2B3iDGtwS